In [ ]:
import numpy as np

##################################################
##### Matplotlib boilerplate for consistency #####
##################################################
from ipywidgets import interact
from ipywidgets import FloatSlider
from matplotlib import pyplot as plt

%matplotlib inline

from IPython.display import set_matplotlib_formats
set_matplotlib_formats('svg')

global_fig_width = 8
global_fig_height = global_fig_width / 1.61803399
font_size = 12

plt.rcParams['axes.axisbelow'] = True
plt.rcParams['axes.edgecolor'] = '0.8'
plt.rcParams['axes.grid'] = True
plt.rcParams['axes.labelpad'] = 8
plt.rcParams['axes.linewidth'] = 2
plt.rcParams['axes.titlepad'] = 16.0
plt.rcParams['axes.titlesize'] = font_size * 1.4
plt.rcParams['figure.figsize'] = (global_fig_width, global_fig_height)
plt.rcParams['font.sans-serif'] = ['Computer Modern Sans Serif', 'DejaVu Sans', 'sans-serif']
plt.rcParams['font.size'] = font_size
plt.rcParams['grid.color'] = '0.8'
plt.rcParams['grid.linestyle'] = 'dashed'
plt.rcParams['grid.linewidth'] = 2
plt.rcParams['lines.dash_capstyle'] = 'round'
plt.rcParams['lines.dashed_pattern'] = [1, 4]
plt.rcParams['xtick.labelsize'] = font_size
plt.rcParams['xtick.major.pad'] = 4
plt.rcParams['xtick.major.size'] = 0
plt.rcParams['ytick.labelsize'] = font_size
plt.rcParams['ytick.major.pad'] = 4
plt.rcParams['ytick.major.size'] = 0
##################################################

## Conditional distributions

### Example Two-dimensional distribution:
- Imagine you are interested in the interrelation between the circumference of a person’s head ($H$) and the volume of their brain ($B$).
- Based on data we find there is a positive correlation between these two variables, which we represent in a distribution $P(H, B)$.

![](fig/Human-brain.SVG)

In [ ]:
from scipy.stats import multivariate_normal

def brain_pdf(xy):
    return multivariate_normal.pdf(xy, mean=[0, 0], cov=[[0.2, 0.1],[0.1,0.2]]); y

def show_brain_distribution(conditional=False):
    x = np.linspace(-1, 1, 50)
    y = np.linspace(-1, 1, 40)

    X, Y = np.meshgrid(x, y)
    XY = np.column_stack([X.flat, Y.flat])

    Z = brain_pdf(XY).reshape(X.shape)
    
    plt.contourf(X, Y, Z, 20, cmap='RdGy');
    
    if conditional:
        plt.plot([-1,1],[])
    plt.xlabel(r'$H$')
    plt.ylabel(r'$B$')
    plt.colorbar()

In [ ]:
show_brain_distribution()

In [ ]:
## Conditional distributions

- **Question:** If an individual has a brain volume of 1450cm3, then what does the distribution for their head circumference look like?
- **Answer:** Use law of conditional probability:

$$P(H|B = 1450) = \frac{P(B = 1450, H)} {P(B = 1450)}$$

- **Analogy:** imagine walking over the probability distribution along a line of B = 1450cm3, and recording your height as you go.